In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas
import pandas as pd
from shapely.geometry import Point
import warnings
from shapely.wkt import loads
import trackintel as ti
from tqdm import tqdm

In [2]:
stop_point_df = pd.read_csv('staypoints.csv')
stop_point_df

,id,user_id,started_at,finished_at,geom
0,0,1,2022-09-05 00:23:55-04:00,2022-09-05 00:30:32-04:00,POINT (-81.6578598100000050 29.6550407400000005)
1,1,1,2022-09-05 00:31:55-04:00,2022-09-05 00:36:45-04:00,POINT (-81.6840837050000061 29.6517493699999974)
2,2,1,2022-09-05 00:38:57-04:00,2022-09-05 00:43:14-04:00,POINT (-81.6840733899999947 29.6517569549999997)
3,3,1,2022-09-05 00:53:25-04:00,2022-09-05 00:57:50-04:00,POINT (-81.6840733899999947 29.6517619549999978)
4,4,1,2022-09-05 01:47:48-04:00,2022-09-05 01:57:58-04:00,POINT (-81.6840667799999949 29.6517639099999997)
...,...,...,...,...,...
587602,587602,26507,2022-10-25 03:06:21-04:00,2022-10-25 03:09:12-04:00,POINT (-81.2044099999999958 29.4846100000000000)
587603,587603,26507,2022-10-25 03:09:12-04:00,2022-10-25 03:17:47-04:00,POINT (-81.2028099999999995 29.4776099999999985)
587604,587604,26507,2022-10-26 22:56:59-04:00,2022-10-26 23:07:58-04:00,POINT (-81.2047730000000030 29.4855580000000010)
587605,587605,26508,2022-10-28 00:47:04-04:00,2022-10-28 00:55:00-04:00,POINT (-81.6688213850000011 30.3685913700000008)


In [3]:
retail_pd = pd.read_csv('retail.csv')
retail_pd

,Unnamed: 0,ObjectId,Store_Name,Address,Address_Li,City,State,Zip5,Zip4,County,Longitude,Latitude,geometry
0,193,194,Sunshine Food Mart #202,1402 NE 25th Ave,NaN,Ocala,FL,34470,4861.0,MARION,-82.103302,29.201019,POINT (-82.10330199999999 29.20101900000003)
1,432,433,Macedonia Store,6128 County Road 23c,NaN,Macclenny,FL,32063,3714.0,BAKER,-82.138069,30.333633,POINT (-82.13806899999997 30.333633000000077)
2,906,907,Kurdi Fresh Seafood Market,452 N Marion Ave,NaN,Lake City,FL,32055,2877.0,COLUMBIA,-82.637283,30.194494,POINT (-82.63728299999997 30.194494000000077)
3,933,934,Union Street Farmers Market,111 E University Ave,NaN,Gainesville,FL,32601,5425.0,ALACHUA,-82.323547,29.651918,POINT (-82.32354699999996 29.651918000000023)
4,997,998,Bell's Produce,1190 S Main St,NaN,Bell,FL,32619,2378.0,GILCHRIST,-82.862099,29.755329,POINT (-82.86209899999994 29.755329000000074)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,244891,244892,Soriya Food Market,3561 Emerson St,NaN,Jacksonville,FL,32207,5628.0,DUVAL,-81.628120,30.291224,POINT (-81.62811999999997 30.291224000000057)
1995,244897,244898,Sunshine Food Mart 248,3495 W Anthony Rd,NaN,Ocala,FL,34475,2674.0,MARION,-82.143723,29.222822,POINT (-82.14372299999997 29.222822000000065)
1996,244899,244900,Star Food Discount,5120 N Pearl St,NaN,Jacksonville,FL,32208,5117.0,DUVAL,-81.656944,30.370920,POINT (-81.65694399999995 30.37092000000007)
1997,245092,245093,Quick N Save #2,1010 S Pine Ave,NaN,Ocala,FL,34471,995.0,MARION,-82.140869,29.178795,POINT (-82.14086899999995 29.178795000000036)


In [4]:
def is_close(dist, latA1, lonA2, latB1, lonB2):
    # Create shapely Point objects from the locations
    point1 = Point(lonA2, latA1)
    point2 = Point(lonB2, latB1)
    
    # Calculate distance between points in meters
    distance = point1.distance(point2) * 111319.9
    
    # Return True if distance is less than or equal to dist meters
    return distance <= dist

In [5]:
count = 0
warnings.simplefilter(action='ignore', category=Warning)
data_to_append_list = []
for sp in tqdm(stop_point_df.itertuples(), total=len(stop_point_df)):
    p = loads(sp[5])
    lat = p.y
    lon = p.x
    
    for row in retail_pd.itertuples():
        if is_close(200, float(row[12]), float(row[11]), lat, lon):
            count += 1
            
            data_to_append = {'id': sp[1], 'user_id': sp[2], 'started_at': sp[3], 'finished_at': sp[4], 'lat': lat, 'lon': lon, 'retail_id': row[1], 
                                'retail_lat': float(row[12]), 'retail_lon': float(row[11])}
            data_to_append_list.append(data_to_append)
            
            break

new_df = pd.DataFrame.from_records(data_to_append_list)
print('Point close to retail: ', count)

100%|██████████| 587607/587607 [7:19:46<00:00, 22.27it/s]  


Point close to retail:  76237


In [6]:
new_df

,id,user_id,started_at,finished_at,lat,lon,retail_id,retail_lat,retail_lon
0,0,1,2022-09-05 00:23:55-04:00,2022-09-05 00:30:32-04:00,29.655041,-81.657860,145695,29.655165,-81.656845
1,5,1,2022-09-05 01:59:44-04:00,2022-09-05 02:04:49-04:00,29.655041,-81.657860,145695,29.655165,-81.656845
2,11,1,2022-09-06 02:05:03-04:00,2022-09-06 02:07:03-04:00,29.655041,-81.657860,145695,29.655165,-81.656845
3,19,1,2022-09-08 01:29:56-04:00,2022-09-08 01:32:11-04:00,29.655036,-81.657865,145695,29.655165,-81.656845
4,22,1,2022-09-10 02:09:19-04:00,2022-09-10 02:11:50-04:00,29.655040,-81.657860,145695,29.655165,-81.656845
...,...,...,...,...,...,...,...,...,...
76232,587551,26497,2022-10-22 02:50:59-04:00,2022-10-22 03:00:35-04:00,29.407710,-82.248080,18329,29.406757,-82.248161
76233,587579,26497,2022-10-30 01:31:03-04:00,2022-10-30 01:33:29-04:00,30.002802,-82.597425,30766,30.001783,-82.597435
76234,587586,26498,2022-10-26 23:04:35-04:00,2022-10-26 23:07:15-04:00,30.322587,-81.557411,58187,30.321955,-81.556618
76235,587588,26504,2022-10-25 20:52:41-04:00,2022-10-25 20:56:31-04:00,29.651005,-82.330360,52510,29.650291,-82.330154


In [7]:
print(len(new_df['user_id'].unique()))

4754


In [8]:
new_df.to_csv('food_inference_200.csv')